In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'TINYAES128C'
SS_VER = 'SS_VER_2_1'

In [2]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [3]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes-bootloader
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

SS_VER set to SS_VER_2_1
rm -f -- simpleserial-bootloader-CWLITEXMEGA.hex
rm -f -- simpleserial-bootloader-CWLITEXMEGA.eep
rm -f -- simpleserial-bootloader-CWLITEXMEGA.cof
rm -f -- simpleserial-bootloader-CWLITEXMEGA.elf
rm -f -- simpleserial-bootloader-CWLITEXMEGA.map
rm -f -- simpleserial-bootloader-CWLITEXMEGA.sym
rm -f -- simpleserial-bootloader-CWLITEXMEGA.lss
rm -f -- objdir-CWLITEXMEGA/*.o
rm -f -- objdir-CWLITEXMEGA/*.lst
rm -f -- bootloader.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes.s aes-independant.s
rm -f -- bootloader.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes.d aes-independant.d
rm -f -- bootloader.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes.i aes-independant.i
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even f

bootloader.c: In function ‘set_addr’:
bootloader.c:65:49: warning: left shift count >= width of type [-Wshift-count-overflow]
     mem_addr = buf[0] | (buf[1] << 8) | (buf[2] << 16) | (buf[3] << 24);
                                                 ^
bootloader.c:65:66: warning: left shift count >= width of type [-Wshift-count-overflow]
     mem_addr = buf[0] | (buf[1] << 8) | (buf[2] << 16) | (buf[3] << 24);
                                                                  ^


In [4]:
fw_path = "../../../hardware/victims/firmware/simpleserial-aes-bootloader/simpleserial-bootloader-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
target.reset_comms()

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 4287 bytes


In [5]:
calc_crc = cw.targets.SimpleSerial2._calc_crc

In [6]:
print(calc_crc([0x00, 0x12]))

def bootloader_calc_crc(buf):
    mac_key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6]
    crc_buf = bytearray(mac_key)
    crc_buf.extend(buf)
    return calc_crc(crc_buf)

51


In [7]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
        
        target.reset_comms()
        
        target.simpleserial_write(0x00, bytearray(range(16)))
        target.simpleserial_wait_ack()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()
        target.reset_comms()
        target.simpleserial_write(0x00, bytearray(range(16)))
        target.simpleserial_wait_ack()

In [8]:
scope.adc.samples = 7000
scope.clock.adc_src = "clkgen_x1"
scope.adc.basic_mode = "falling_edge"

In [9]:
target.simpleserial_write(0x00, bytearray(range(16)))
target.simpleserial_wait_ack()

bytearray(b'\x00')

In [10]:
from tqdm.notebook import trange
import chipwhisperer as cw
#Do glitch loop
N = 5000
trace_array = []
textin_array = []
real_text = []
ktp = cw.ktp.Basic()

project = cw.create_project("projects/test_cpa", overwrite=True)

for i in trange(N, desc='Capruring traces'):
    key, text = ktp.next()
    real_text = text[:]
    text.extend([bootloader_calc_crc(text)])
    # print(text)
    scope.arm()
    target.simpleserial_write("p", text)
    scope.capture()
    val = target.simpleserial_wait_ack() # For loop check
    if val[0] != 0x00:
        print('Invalid!!!')
        continue
    trace_array.append(scope.get_last_trace())
    textin_array.append(real_text)
    trace = cw.Trace(scope.get_last_trace(), real_text, bytearray([0]*16), key)
    project.traces.append(trace)
    #print(bytearray(val['full_response'].encode('latin-1')))
    # print(val)
    # cw.plot(scope.get_last_trace())

Capruring traces:   0%|          | 0/5000 [00:00<?, ?it/s]

In [11]:
sbox=[
0x52,0x09,0x6a,0xd5,0x30,0x36,0xa5,0x38,0xbf,0x40,0xa3,0x9e,0x81,0xf3,0xd7,0xfb,
0x7c,0xe3,0x39,0x82,0x9b,0x2f,0xff,0x87,0x34,0x8e,0x43,0x44,0xc4,0xde,0xe9,0xcb,
0x54,0x7b,0x94,0x32,0xa6,0xc2,0x23,0x3d,0xee,0x4c,0x95,0x0b,0x42,0xfa,0xc3,0x4e,
0x08,0x2e,0xa1,0x66,0x28,0xd9,0x24,0xb2,0x76,0x5b,0xa2,0x49,0x6d,0x8b,0xd1,0x25,
0x72,0xf8,0xf6,0x64,0x86,0x68,0x98,0x16,0xd4,0xa4,0x5c,0xcc,0x5d,0x65,0xb6,0x92,
0x6c,0x70,0x48,0x50,0xfd,0xed,0xb9,0xda,0x5e,0x15,0x46,0x57,0xa7,0x8d,0x9d,0x84,
0x90,0xd8,0xab,0x00,0x8c,0xbc,0xd3,0x0a,0xf7,0xe4,0x58,0x05,0xb8,0xb3,0x45,0x06,
0xd0,0x2c,0x1e,0x8f,0xca,0x3f,0x0f,0x02,0xc1,0xaf,0xbd,0x03,0x01,0x13,0x8a,0x6b,
0x3a,0x91,0x11,0x41,0x4f,0x67,0xdc,0xea,0x97,0xf2,0xcf,0xce,0xf0,0xb4,0xe6,0x73,
0x96,0xac,0x74,0x22,0xe7,0xad,0x35,0x85,0xe2,0xf9,0x37,0xe8,0x1c,0x75,0xdf,0x6e,
0x47,0xf1,0x1a,0x71,0x1d,0x29,0xc5,0x89,0x6f,0xb7,0x62,0x0e,0xaa,0x18,0xbe,0x1b,
0xfc,0x56,0x3e,0x4b,0xc6,0xd2,0x79,0x20,0x9a,0xdb,0xc0,0xfe,0x78,0xcd,0x5a,0xf4,
0x1f,0xdd,0xa8,0x33,0x88,0x07,0xc7,0x31,0xb1,0x12,0x10,0x59,0x27,0x80,0xec,0x5f,
0x60,0x51,0x7f,0xa9,0x19,0xb5,0x4a,0x0d,0x2d,0xe5,0x7a,0x9f,0x93,0xc9,0x9c,0xef,
0xa0,0xe0,0x3b,0x4d,0xae,0x2a,0xf5,0xb0,0xc8,0xeb,0xbb,0x3c,0x83,0x53,0x99,0x61,
0x17,0x2b,0x04,0x7e,0xba,0x77,0xd6,0x26,0xe1,0x69,0x14,0x63,0x55,0x21,0x0c,0x7d]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]

In [12]:
HW = [bin(i).count("1") for i in range(256)]

In [13]:
import chipwhisperer.analyzer as cwa
key = bytearray(cwa.aes_funcs.key_schedule_rounds(key, 0, 10))
key = list(key)
print(key)

[208, 20, 249, 168, 201, 238, 37, 137, 225, 63, 12, 200, 182, 99, 12, 166]


In [14]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import brewer

import numpy as np 

output_notebook()
p = figure(title="Ploting diff between trace average per hamming weight group and trace average overall.")

bnum = 0

hw_traces = [[] for _ in range(9)]
avg_overall = []
color_mapper = brewer['PRGn'][9]

for tnum in range(len(trace_array)):
    hw_sel_group = HW[aes_internal(textin_array[tnum][bnum], key[bnum])]
    hw_traces[hw_sel_group].append(trace_array[tnum])
hw_average = np.array([np.average(hw_traces[group], axis=0) for group in range(9)])
avg_overall = np.average(hw_average, axis=0)

print(np.shape(hw_average))
xrange = range(len(trace_array[0]))

for hw in range(9):
    p.line(xrange, (hw_average[hw] - avg_overall), line_color=color_mapper[hw])
show(p)

Loading BokehJS ...

(9, 7000)


In [15]:
sbox_loc = np.argmax(abs(hw_average[0] - avg_overall))
print(sbox_loc)

6157


In [16]:
output_notebook
p = figure(title="HW vs Voltage Measurement")

p.line(range(0,9), hw_average[:,sbox_loc], line_color='red')
p.xaxis.axis_label = "Hamming Weight of Intermediate Value"
p.yaxis.axis_label = "Average Value of Measurement"
show(p)
print(np.shape(hw_average[:,sbox_loc]))

(9,)


In [17]:
target.dis()
scope.dis()

True